In [ ]:
import numpy as np
import xarray as xr
import scipy.stats as st
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import requests
import os,errno
import sys
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER,LATITUDE_FORMATTER
import matplotlib.ticker as mticker
import cartopy.feature as cfeature
import datetime as dt
import pandas as pd
import time
import urllib.request
import metpy.calc as mpcalc
import salem

dir_data='/Users/ahenny/'
dir1='/Volumes/LaCie/'
dir2='/Volumes/Extreme Pro/'

In [ ]:
ds=xr.open_dataset('/Users/ahenny/Downloads/full_data_daily_v2022_10_'+str(1983)+'.nc')
precip=ds['precip'].transpose('time','lat','lon')[0,:,:]

persiann_threshold_final=xr.zeros_like(precip)
precip.close()
ds.close()

dk=xr.Dataset()
dk['threshold']=(('lat','lon'),persiann_threshold_final.values)
dk.coords['lat']=persiann_threshold_final.lat
dk.coords['lon']=persiann_threshold_final.lon

try:
    os.remove(dir_data+'gpcc_threshold_r95p.nc')
except OSError:
    pass
dk.to_netcdf(dir_data+'gpcc_threshold_r95p.nc',mode='w',format='NETCDF4')

dk.close()
persiann_threshold_final.close()

In [ ]:
yrs=np.arange(1991,2021,1)

lon_bins=np.arange(-180,210,30)
lat_bins=np.arange(-90,120,30)
        
dates_all=[]
for i in range(len(yrs)):
    year=yrs[i]
    year_length=365
    if year%4==0:
        year_length=366
            
    date_start=dt.datetime(year,1,1,0)
    date_range=[date_start+dt.timedelta(days=x) for x in range(year_length)]
    dates_all=dates_all+date_range

for j in range(len(lat_bins)-1):
    for k in range(len(lon_bins)-1):
        print((j,k))
        ds1=xr.open_dataset(dir_data+'gpcc_threshold_r95p.nc')
        threshold=ds1['threshold'].values
        
        lat_range=[x for x in ds1.lat.values if lat_bins[j]<=x<lat_bins[j+1]]
        lon_range=[x for x in ds1.lon.values if lon_bins[k]<=x<lon_bins[k+1]]
        
        zipped_lat=list(zip(np.arange(ds1.lat.size),ds1.lat.values.tolist()))
        zipped_lon=list(zip(np.arange(ds1.lon.size),ds1.lon.values.tolist()))

        select_lat_indices=[x[0] for x in zipped_lat if x[1] in lat_range]
        select_lon_indices=[x[0] for x in zipped_lon if x[1] in lon_range]
            
        min_lat_index=min(select_lat_indices)
        min_lon_index=min(select_lon_indices)
        max_lat_index=max(select_lat_indices)+1
        max_lon_index=max(select_lon_indices)+1
        
        threshold_test=np.sum(threshold[min_lat_index:max_lat_index,min_lon_index:max_lon_index])
        
        if threshold_test==0:
        
            for i in range(len(yrs)):
                year=yrs[i]
                #print(year)

                ds=xr.open_dataset('/Users/ahenny/Downloads/full_data_daily_v2022_10_'+str(year)+'.nc')
                precip=ds['precip'].transpose('time','lat','lon')
                precip=precip.sel(lat=lat_range,lon=lon_range).values

                if i==0:
                    precip_concat=precip
                else:
                    precip_concat=np.concatenate((precip_concat,precip),axis=0)    

            precip_mask=np.ones_like(precip_concat)
            precip_mask=np.where(precip_concat>=0,precip_mask,np.nan)
            precip_mask=np.nansum(precip_mask,axis=0)
            precip_mask=np.where(precip_mask>=0.75*len(precip_concat[:,0,0]),1,np.nan)
            
            precip_concat=np.where(precip_concat>=1.,precip_concat,np.nan)
            threshold_array=np.nanquantile(precip_concat,0.95,axis=0)
            threshold_array=np.where(precip_mask==1,threshold_array,np.nan)
            threshold[min_lat_index:max_lat_index,min_lon_index:max_lon_index]=threshold_array#.values

            dk=xr.Dataset()
            dk['threshold']=(('lat','lon'),threshold)
            dk.coords['lat']=ds1.lat
            dk.coords['lon']=ds1.lon

            try:
                os.remove(dir_data+'gpcc_threshold_r95p.nc')
            except OSError:
                pass
            dk.to_netcdf(dir_data+'gpcc_threshold_r95p.nc',mode='w',format='NETCDF4')
            dk.close()
            ds.close()
            ds1.close()